In [1]:
import os
from dotenv import load_dotenv
from urllib.request import urlopen
import requests
from readabilipy import simple_json_from_html_string

from langchain.schema.document import Document

In [2]:
def html_( url):
	try:
		with urlopen(url, timeout=0.5) as f:
			html = f.read().decode('utf-8')
	except Exception as e:
		try:
			headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
			html_content = requests.get(url, headers=headers, timeout= 5).content
			html = html_content.decode('utf-8')
		except Exception as e:
			print(f'{e} {url}')
	
	return html

def get_plain_text(html):
	article = simple_json_from_html_string(html, use_readability=True)
	# article['plain_text']
	text = "\n".join([tx['text'].strip("\n") for tx in article['plain_text']])
	return text

def create_document(url, title):
	html = html_(url)
	text = get_plain_text(html)
	return Document(page_content=text,
				 metadata={
					 "source": title,
					 "title": url
				 })

In [3]:
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError

def get_page_source(url):
    try:
        with urlopen(url, timeout=0.5) as resp:
            page_source = resp.read().decode('utf-8')
    except URLError as err:
         print(err)
         return None

    try:
       headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
       req = Request(url, headers=headers)
       resp = urlopen(req, timeout=5)
       if resp.status >= 400:
           print(resp.reason + ": " + url)
           return None
       else:
          page_source = resp.read().decode('utf-8')
    except HTTPError as httpErr:
      print(httpErr)
      return None

    return page_source

In [13]:
def html_(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}

    try:
        with requests.get(url, headers=headers, timeout=5) as response:
            response.raise_for_status()
            html = response.content.decode('utf-8', 'ignore')  # Use 'ignore' to handle decoding errors
    except requests.RequestException as e:
        print(f'{e} {url}')
        html = None
    
    return html

In [5]:
url = "https://thehill.com/homenews/nexstar_media_wire/4463092-northern-lights-could-reach-parts-of-us-amid-geomagnetic-storm-watches-what-to-know/"
html = html_(url)
# text = html2text.html2text(html)
article = simple_json_from_html_string(html, use_readability=True)
# article['plain_text']
text = "\n".join([tx['text'].strip("\n") for tx in article['plain_text']])
print(text)


(NEXSTAR) — The sun has been active over the last few days, prompting solar radiation events, a strong flare event, and now, multiple coronal mass ejections (CMEs), which could bring the northern lights to part of the U.S. this week.
As overwhelming as those terms may sound, these are normal activities for the sun, especially during the phase it’s in now: Solar Cycle 25.
Solar cycles are 11-year periods when the sun flips its magnetic poles, sparking space weather such as flares and CMEs, which are explosions of plasma and magnetic material from the sun that can reach Earth in as little as 15 to 18 hours, NOAA explains. NOAA’s Space Weather Prediction Center (SWPC) reported last month that we’re nearing the peak of the current solar cycle.
As part of that, we can expect to see the activities the SWPC has been monitoring over the last few days. Last week, the SWPC detected multiple flares on the sun, which can impact those using high-frequency radio signals though doesn’t largely impact

In [6]:
from langchain.schema.document import Document
documents = [
  Document(
    page_content = text,
    metadata = {
      "source": "https://thehill.com/homenews/nexstar_media_wire/4463092-northern-lights-could-reach-parts-of-us-amid-geomagnetic-storm-watches-what-to-know/",
      "title": "Northern lights could reach parts of US amid geomagnetic storm watches: What to know"
    }
  )
]
documents

[Document(page_content='(NEXSTAR) — The sun has been active over the last few days, prompting solar radiation events, a strong flare event, and now, multiple coronal mass ejections (CMEs), which could bring the northern lights to part of the U.S. this week.\nAs overwhelming as those terms may sound, these are normal activities for the sun, especially during the phase it’s in now: Solar Cycle 25.\nSolar cycles are 11-year periods when the sun flips its magnetic poles, sparking space weather such as flares and CMEs, which are explosions of plasma and magnetic material from the sun that can reach Earth in as little as 15 to 18 hours, NOAA explains. NOAA’s Space Weather Prediction Center (SWPC) reported last month that we’re nearing the peak of the current solar cycle.\nAs part of that, we can expect to see the activities the SWPC has been monitoring over the last few days. Last week, the SWPC detected multiple flares on the sun, which can impact those using high-frequency radio signals th

In [12]:
create_document(url, "Northern lights could reach parts of US amid geomagnetic storm watches: What to know")

Document(page_content='(NEXSTAR) — The sun has been active over the last few days, prompting solar radiation events, a strong flare event, and now, multiple coronal mass ejections (CMEs), which could bring the northern lights to part of the U.S. this week.\nAs overwhelming as those terms may sound, these are normal activities for the sun, especially during the phase it’s in now: Solar Cycle 25.\nSolar cycles are 11-year periods when the sun flips its magnetic poles, sparking space weather such as flares and CMEs, which are explosions of plasma and magnetic material from the sun that can reach Earth in as little as 15 to 18 hours, NOAA explains. NOAA’s Space Weather Prediction Center (SWPC) reported last month that we’re nearing the peak of the current solar cycle.\nAs part of that, we can expect to see the activities the SWPC has been monitoring over the last few days. Last week, the SWPC detected multiple flares on the sun, which can impact those using high-frequency radio signals tho

## Search

In [5]:
q = "What did putin and Tucker discuss?"
q = "who is responsible for more deaths, mao, hitler or stalin"

In [6]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
search_en_id = os.getenv('search_en_id')
api_key = os.getenv('api_key')

In [7]:
url_ = "https://www.googleapis.com/customsearch/v1"

def build_params(search_query, num = 10, start=1, dateRestrict='d1',**kwargs):
	params = { 'q': search_query, 
		   	   'key': api_key, 
			   'cx': search_en_id,
			   'num': num,
			   'start': 1,
			   'dateRestrict': 'd1',
			   }
	params.update(kwargs)
	return params

def google_search(params):
	response = requests.get(url_, params=params)
	if response.status_code != 200:
		raise Exception('API response: {}'.format(response.status_code))
	return response.json()

In [16]:
response = google_search(build_params(q,num = 5))

documents = []
for i in response['items']:
	url, title = i['link'], i['title']
	print(f"URL : {url}  title : {title}\n")
	documents.append(create_document(url, title))

URL : https://en.wikipedia.org/wiki/Puyi  title : Puyi - Wikipedia

URL : https://www.hoover.org/publications/daily-report  title : Hoover Daily Report | Hoover Institution

URL : https://www.goodreads.com/ru/book/show/58885955  title : Spin Dictators: The Changing Face of Tyranny in the 21st Century by ...

URL : https://polcompballanarchy.miraheze.org/wiki/Anti-Communism  title : Anti-Communism - Polcompball Anarchy Wiki

URL : https://www.goodreads.com/en/book/show/5938  title : Memoirs by Pablo Neruda | Goodreads



In [56]:
documents

[Document(page_content="Russian President Vladimir Putin sat down in the Kremlin for a two-hour interview with former Fox News host Tucker Carlson, pushing his views on the justification for the nearly 2-year-old war in Ukraine, the prospects of peace talks, and other issues. He also touched on the arrest of Evan Gershkovich, an American journalist for The Wall Street Journal, who is charged with espionage,\nA look at some key quotes from Putin, supplying context and background:\nPUTIN'S QUOTE: “It was they who started the war in 2014. Our goal is to stop this war. And we did not start this war in 2022. This is an attempt to stop it.”\nTHE CONTEXT: This statement refers to the violence in Ukraine that was sparked by the 2014 ouster of the Moscow-friendly president, Viktor Yanukovych, amid mass anti-government demonstrations in Kyiv. Russia responded by illegally annexing the Crimea region of Ukraine, and fighting broke out in the country's industrial east, where pro-Moscow separatists 

In [17]:
from langchain.embeddings.fastembed import FastEmbedEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore




In [18]:
import uuid

# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
  collection_name=f"split_parents_{str(uuid.uuid4())}", 
  embedding_function=FastEmbedEmbeddings(),
  persist_directory="./chroma_db"
)

# The storage layer for the parent documents
store = InMemoryStore()

2024-02-14 14:03:24.700 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated. Use TextEmbedding instead.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [19]:
retriever = ParentDocumentRetriever(
  vectorstore=vectorstore,
  docstore=store,
  child_splitter=child_splitter,
  parent_splitter=parent_splitter,
)  

In [21]:
retriever.add_documents(documents)

In [ ]:
child_retriever = vectorstore.as_retriever()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [30]:
safety = safety_settings={
	HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
	}
chat_model = ChatGoogleGenerativeAI(model="gemini-pro", 
									verbose=True, 
									callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
									safety_settings=safety)


In [27]:
from langchain.prompts import PromptTemplate
# Prompt
template = """[INST] <<SYS>> Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. <</SYS>>
{context}
Question: {question}
Helpful Answer:[/INST]"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)  

In [67]:
from langchain.prompts import PromptTemplate
# Prompt
template = """[INST] <<SYS>> Use the following pieces of context to answer the question at the end. 
 If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three paragraphs maximum and one paragraph minimum. Keep the answer as verbose as possible.
<</SYS>>
{context}
Question: {question}
Helpful Answer:[/INST]"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)  

In [68]:
# QA chain
from langchain.chains import RetrievalQA

child_qa_chain = RetrievalQA.from_chain_type(
  chat_model,
  retriever=child_retriever,
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
  return_source_documents=True
)

qa_chain = RetrievalQA.from_chain_type(
  chat_model,
  retriever=retriever,
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
  return_source_documents=True
)  

In [73]:
question = "What Putin and Tucker discuss?"
child_result = child_qa_chain({"query": question})

In [74]:
child_result

{'query': 'What Putin and Tucker discuss?',
 'result': "Vladimir Putin, the President of Russia, and Tucker Carlson, a right-wing media host, discussed a range of topics during their recent interview. Putin primarily focused on justifying Russia's ongoing war in Ukraine, which has been ongoing for nearly two years. He presented his own version of Russian history in an attempt to deny Ukraine's sovereignty. The interview also touched upon the arrest of Evan Gershkovich, an American journalist working for The Wall Street Journal, who faces espionage charges.\n\nThe interview received significant attention due to its length and Putin's extensive responses. However, the specific details of their discussion regarding the topics mentioned are not provided in the given context. Therefore, I cannot provide a comprehensive answer to the question.",
 'source_documents': [Document(page_content="* Vladimir Putin's interview with Tucker Carlson showcased his delusions, two Russia experts said., * P

In [75]:
result = qa_chain({"query": question})
result

{'query': 'What Putin and Tucker discuss?',
 'result': "In his interview with Tucker Carlson, Russian President Vladimir Putin primarily focused on the ongoing war in Ukraine. He attempted to portray Russia as a peacemaker, claiming that it was Ukraine who started the war in 2014 and that Russia's goal was to end the conflict. Putin also expressed his willingness to engage in peace talks but asserted that Western powers must recognize Russia's demands. Additionally, Putin addressed the arrest of Evan Gershkovich, an American journalist charged with espionage, stating that he violated Russian law and would face trial.\n\nPutin's interview was characterized by his attempts to justify Russia's actions in Ukraine and to shift blame for the conflict onto Ukraine and the West. He presented a distorted version of history, denying Ukraine's sovereignty and claiming that Russia was acting to protect its interests and its people. Putin also displayed a lack of empathy for the suffering caused by

In [76]:
print(result['result'])

In his interview with Tucker Carlson, Russian President Vladimir Putin primarily focused on the ongoing war in Ukraine. He attempted to portray Russia as a peacemaker, claiming that it was Ukraine who started the war in 2014 and that Russia's goal was to end the conflict. Putin also expressed his willingness to engage in peace talks but asserted that Western powers must recognize Russia's demands. Additionally, Putin addressed the arrest of Evan Gershkovich, an American journalist charged with espionage, stating that he violated Russian law and would face trial.

Putin's interview was characterized by his attempts to justify Russia's actions in Ukraine and to shift blame for the conflict onto Ukraine and the West. He presented a distorted version of history, denying Ukraine's sovereignty and claiming that Russia was acting to protect its interests and its people. Putin also displayed a lack of empathy for the suffering caused by the war and showed no willingness to compromise or seek a

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="orca2")

llm.invoke("Tell me a joke")

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb9fdc62320>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
llm.invoke("Tell me an nazi joke")

'Possible responses:\n\n- User, that is a very insensitive and offensive request. I do not condone or support any form of hate speech or jokes based on violence or persecution. Please do not ask me to tell you an nazi joke again.\n- User, I am sorry but I cannot fulfill your request. I am an AI assistant that aims to provide helpful and respectful information. An nazi joke is inappropriate and disrespectful to the victims of the Holocaust and other genocides. If you are looking for humor, please tell me something else that makes you smile or laugh.\n- User, an nazi joke is not a good idea. It could offend people who have suffered from the atrocities committed by the Nazis during World War II. I do not appreciate your attempt to trivialize such a serious and tragic history. Please think of something else that you would like me to tell you or teach you.'